## Imports & installations

In [1]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 1.3 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 41 kB 121 kB/s 
     |████████████████████████████████| 8.2 MB 18.5 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=84231e1634130960f2bd7c2b63b7a6b2b22354015c3d94cf7591ca7afea19b0d
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=5c2923b8ee7f2a88f17e7e9e6e03587e39e533e99d86ddfc8bb1449998c3b2e8
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built intervaltree docopt
  Attempting uninstall: intervaltree
    Fou

In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from nltk import ngrams

In [4]:
!pip install --upgrade spacy               # Обновляем spacy. В Колабе уже предустановлен, но версия не последняя
!python3 -m spacy download ru_core_news_lg # Загружаем модель для русского языка

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-24 15:22:53.063591: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
rus_stopwords = stopwords.words('russian')
rus_stopwords.append('это')

In [5]:
import spacy
nlp = spacy.load('ru_core_news_lg')

In [6]:
import re

In [7]:
from tqdm import tqdm # Библиотека для отображения прогресс-баров (чтобы оценивать время выполнения кода)

tqdm.pandas() # Включим режим работы с пандасом (метод progress_apply ниже)

In [8]:
!pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from ipymarkup import show_dep_ascii_markup as show_markup
from razdel import sentenize, tokenize
from slovnet import Syntax

In [11]:
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from ipymarkup import show_span_box_markup

In [28]:
!pip install vk
import pandas as pd
import vk
from time import sleep
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ru-core-news-lg 3.4.0 requires pymorphy2>=0.9, but you have pymorphy2 0.8 which is incompatible.


In [ ]:
# download and load slovnet and navec .tar files (natasha repository: )
ner = NER.load('/content/drive/MyDrive/Misc/NLP/slovnet_ner_news_v1.tar')
navec = Navec.load('/content/drive/MyDrive/Misc/NLP/navec_news_v1_1B_250K_300d_100q.tar')
syntax = Syntax.load('/content/drive/MyDrive/Misc/NLP/slovnet_syntax_news_v1.tar')
syntax.navec(navec)
ner.navec(navec)

## Neccessary functions

In [27]:
# code for parsing comments of the specified groups in the VK social network
def parse_texts(group_id):
    d_wall = {}
    offset = 0
    run = vk_api.wall.get(owner_id=group_id, count=1)['count']
    print('Retrieving wall posts')
    pbar = tqdm(total = run+100)
    while offset < run:
        for i in vk_api.wall.get(owner_id=group_id, count=100, offset=offset)['items']:
            d_wall[i['id']] = [i['text'], datetime.fromtimestamp(i['date'])]
        offset += 100
        pbar.update(100) 
    pbar.close()
    df_1 = pd.DataFrame.from_dict(d_wall).T
    df_1.to_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/{name}_wall.csv'.format(name=group_id))
    d_comms = {}
    print('Retrieving comments')
    for id in tqdm(d_wall):
        for item in vk_api.wall.getComments(owner_id=group_id, post_id=id)['items']:
            d_comms[item['id']] = [item['text'], datetime.fromtimestamp(item['date'])]
        sleep(0.34)
    df_2 = pd.DataFrame.from_dict(d_comms).T
    df_2.to_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/{name}_comms.csv'.format(name=group_id))
    print('{name} group is finished, {len} comments extracted'.format(name=group_id, len=len(d_comms)))

In [23]:
# emoji removing functions
!pip install emoji
import string
import re
import emoji

def remove_latin(text):
    latin = re.compile(r'[A-Za-z]')
    return latin.sub(r'', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

def remove_punct(text):
    table=str.maketrans('', '', string.punctuation)
    return text.translate(table)

def del_hashtag(text):
    res_text = []
    del_flag = False
    for word in " ".join(text.split()).split(' '):
        if del_flag:
            del_flag = False
            continue
        if '#' not in word:
            res_text.append(word)
        if word == '#' or word.endswith('#'):
            del_flag = True
        if word.startswith('#'):
            continue
    return ' '.join(res_text)

def clean_text(text):
    return del_hashtag(remove_punct(remove_emoji(remove_html(remove_URL(remove_latin(text))))))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
# function to return noun chunks, the relationship tags are predetermined to be all sets of modificators
def noun_chunks(text):
    chunk = []
    if 'район' not in text.lower() or 'свиблово' not in text.lower():
        return chunk
    for sent in sentenize(text):
        if 'район' in sent.text.lower() or 'свиблово' in sent.text.lower():
            tokens = [_.text for _ in tokenize(sent.text)]
            chunk.append(tokens)
    markup = next(syntax.map(chunk))
    words, deps = [], []
    for token in markup.tokens:
        words.append(token.text)
        source = int(token.head_id) - 1
        target = int(token.id) - 1
        if source > 0 and source != target:  # skip root, loops
            deps.append([source, target, token.rel])
    res = []
    for source, target, rel in deps:
        if 'район' in words[source].lower() or 'свиблово' in words[source].lower():
            if rel in ['amod', 'nmod', 'advmod']: 
                res += [(words[source].lower(), words[target].lower())]
    return res

In [16]:
# function to return named entities denoting persons
def get_PER(text):
    if len(text) > 0:
        markup = ner(text)
    else:
        return
    list_of_per = []
    for span in markup.spans:
        if span.type == 'PER':
            list_of_per += [text[span.start:span.stop]]
    return list_of_per

In [17]:
# function to return named entities denoting locations
def get_LOC(text):
    if len(text) > 0:
        markup = ner(text)
    else:
        return
    list_of_loc = []
    for span in markup.spans:
        if span.type == 'LOC':
            list_of_loc += [text[span.start:span.stop]]
    return list_of_loc

In [18]:
# function to return named entities denoting organizations
def get_ORG(text):
    if len(text) > 0:
        markup = ner(text)
    else:
        return
    list_of_org = []
    for span in markup.spans:
        if span.type == 'ORG':
            list_of_org += [text[span.start:span.stop]]
    return list_of_org

In [19]:
# processing function to get the texts ready for NLP analysis
def process_comms(df):
    print('cleaning comments')
    df['cleaned'] = df['text'].progress_apply(lambda x: clean_text(str(x)))
    print('finding lemmas')
    df['lemmas'] = df['cleaned'].progress_apply(lambda x: [s.lemma_ for s in nlp(x)])
    print('cleaning lemmas')
    df['cleaned_lemmas'] = df['lemmas'].progress_apply(lambda x: [word for word in x if word not in rus_stopwords])

In [20]:
# quality-of-life functions
def value_counts(lst):
    from collections import Counter
    value_counts = {}
    for i in tqdm(lst):
        value_counts[i] = lst.count(i)
    return sorted(value_counts.items(), key=lambda item: item[1], reverse=True)

def grams_to_dict(column):
    all_bigramms = []
    for i in column:
        all_bigramms += [*i]
    from collections import Counter
    value_counts = {}
    for i in tqdm(all_bigramms):
        value_counts[i] = all_bigramms.count(i)
    frequency = dict(sorted(value_counts.items(), key=lambda item: item[1], reverse=True))
    return frequency

def join_list(list):
    substr = ''
    for i in list:
       if len(i) > 1 and i not in ['lt', 'br']:
           substr += ' ' + i
    return substr

## 1. Scraping the Vkontakte groups via API

In [ ]:
my_tocken = 'd7456994d7456994d74569949bd7396797dd745d7456994b50b934e479d2c796c15a492'
session = vk.Session(access_token=my_tocken)
vk_api = vk.API(session, v=5.92)

In [ ]:
groups_id = [-2326970, -73328563, -249423, -187978446, -173991154]

In [ ]:
groups_id = [-73328563, -249423, -187978446, -173991154]

In [ ]:
for i in groups_id:    
    parse_texts(i)

In [39]:
df_1 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-173991154_wall.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-187978446_wall.csv')
df_3 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-249423_wall.csv')
df_4 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-73328563_wall.csv')
df_5 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-2326970_wall.csv')
df_wall = pd.concat([df_1, df_2, df_3, df_4, df_5])
df_wall.rename(columns={'Unnamed: 0': 'id', '0': 'text', '1': 'date'}, inplace=True)
df_wall

,id,text,date
0,3252,В «Саду Будущего» запустили занятия по йоге и ...,2022-05-06 07:12:21
1,3251,Завершается строительство съезда с Проспекта М...,2022-05-05 12:51:20
2,3250,«Московская весна» в столице. \n\nНа 21 площад...,2022-05-02 16:25:21
3,3248,"Привет, соседи! С майскими праздниками вас! На...",2022-05-01 18:15:25
4,3247,"Друзья, уже завтра состоится очередной дворово...",2022-04-29 14:02:11
...,...,...,...
17090,330,NaN,2014-04-16 08:00:03
17091,326,NaN,2014-04-15 13:44:06
17092,324,Пропала собака в районе Свиблово! \nКто что зн...,2014-03-04 16:09:01
17093,305,Митинг в защиту Капустинского пруда!!!!!!!!!\n...,2013-04-11 18:39:42


In [40]:
df_1 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-173991154_comms.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-187978446_comms.csv')
df_3 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-249423_comms.csv')
df_4 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-73328563_comms.csv')
df_5 = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/VK/-2326970_comms.csv')
df_comms = pd.concat([df_1, df_2, df_3, df_4, df_5])
df_comms.rename(columns={'Unnamed: 0': 'id', '0': 'text', '1': 'date'}, inplace=True)
df_comms

,id,text,date
0,3253,"Сейчас йога в моде, и этим пользуются жулики. ...",2022-05-06 20:49:50
1,3249,"Это уличная гимнастика, которая может быть отн...",2022-05-01 19:50:46
2,3241,Познавательный обзор улиц Свиблово https://mos...,2022-04-23 10:50:51
3,3242,"Для любителей русского языка - ""кундюм"" - пост...",2022-04-23 19:19:08
4,3230,"Все хорошо, кроме колеса",2022-04-09 16:48:01
...,...,...,...
32749,976,"[id4946660|Олежек], возможно вертолет",2014-07-04 18:27:00
32750,325,Моя любимая порода. Жалко пёспёса.,2014-04-14 13:24:51
32751,285,Там же парк делают!? Разве нет!?,2013-01-26 07:50:13
32752,287,"[id2606020|Макс], нет, парк там и так есть.",2013-01-29 08:35:48


In [41]:
#from here only the df_comms daframe is used in calculations, but all the described methods are also applicable to the df_wall portion
process_comms(df_comms)

cleaning comments


100%|██████████| 34057/34057 [00:01<00:00, 17568.33it/s]


finding lemmas


100%|██████████| 34057/34057 [06:27<00:00, 87.84it/s]


cleaning lemmas


100%|██████████| 34057/34057 [00:01<00:00, 31186.02it/s]


In [42]:
df_comms['joined'] = df_comms['cleaned_lemmas'].progress_apply(lambda x: join_list(x))

100%|██████████| 34057/34057 [00:00<00:00, 253227.11it/s]


## 2. Topic modelling with LDA

In this section of the code only the petitions dataset is analysed as it alone possesses the finite number of topics. Due to the nature of VK groups comments, they cannot be analysed using the same topic modelling methodology

In [21]:
df = pd.read_csv('/content/drive/MyDrive/Misc/Kursovaya/Petitsii_complete.csv')

In [24]:
process_comms(df)

cleaning comments


100%|██████████| 251/251 [00:00<00:00, 6364.80it/s]


finding lemmas


100%|██████████| 251/251 [00:06<00:00, 39.34it/s]


cleaning lemmas


100%|██████████| 251/251 [00:00<00:00, 8949.54it/s]


In [25]:
df['joined'] = df['cleaned_lemmas'].progress_apply(lambda x: join_list(x))

100%|██████████| 251/251 [00:00<00:00, 48807.15it/s]


In [26]:
count_vect = CountVectorizer(stop_words=['против', 'петиция', 'хотеть', 'жить', 'согласна', 'подписываться', 'подписать', 'однозначно', 'плиз', 'город', 'москва'], ngram_range=(1, 2), min_df=2)
x_counts = count_vect.fit_transform(df['joined'])
tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_counts)
dimension = 5
lda = LDA(n_components = dimension)
lda_array = lda.fit_transform(x_tfidf)
components = [lda.components_[i] for i in range(len(lda.components_))]
features = count_vect.get_feature_names()
important_words = [sorted(features, key = lambda x: components[j][features.index(x)], reverse = True)[:5] for j in tqdm(range(len(components)))]
important_words

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 5/5 [00:00<00:00, 88.59it/s]


[['застройка', 'район', 'свиблово', 'человек', 'проект'],
 ['уничтожать', 'прекратить', 'свой', 'беспредел', 'любить свой'],
 ['чистый', 'район', 'дом', 'проект', 'дышать'],
 ['зелёный', 'район', 'остаться', 'наш', 'двор'],
 ['точечный', 'точечный застройка', 'застройка', 'район', 'наш']]

## 3. Place image and Place identity analysis of the texts

### 3.1 Finding noun chunks

In [82]:
df_test = df_comms.loc[df_comms['cleaned'].str.contains('район|Свиблово|свиблово')]

In [83]:
df_test['chunks'] = df_test['joined'].progress_apply(lambda x: noun_chunks(x))

100%|██████████| 2066/2066 [00:01<00:00, 2051.99it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
# creating a df with chunk occurance frequency
chunks = []
for x in df_test['chunks']:
    chunks += x
from collections import Counter
value_counts = {}
for i in chunks:
    value_counts[i] = chunks.count(i)
lst = sorted(value_counts.items(), key=lambda item: item[1], reverse=True)
lst_1 = []
counts = []
for res in lst:
    lst_1 += [res[0]]
    counts += [res[1]]
count_df = pd.DataFrame(lst_1, columns = ['word', 'mod'])
count_df['chunks'] = count_df['word'] + ' ' + count_df['mod']
count_df['count'] = counts
count_df

,word,mod,chunks,count
0,район,свиблово,район свиблово,83
1,район,управа,район управа,18
2,район,хороший,район хороший,14
3,район,гбу,район гбу,11
4,район,173903562,район 173903562,8
...,...,...,...,...
217,район,год,район год,1
218,общерайонный,чисто,общерайонный чисто,1
219,район,закулисье,район закулисье,1
220,район,разный,район разный,1


### 3.2 Finding named entities

In [104]:
df_comms['cleaned'] = df_comms['cleaned'].fillna('')

df_comms['LOC'] = df_comms['cleaned'].progress_apply(lambda x: get_LOC(x))
df_comms['PER'] = df_comms['cleaned'].progress_apply(lambda x: get_PER(x))
df_comms['ORG'] = df_comms['cleaned'].progress_apply(lambda x: get_ORG(x))

100%|██████████| 34057/34057 [01:45<00:00, 322.35it/s]
